In [8]:

from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from langchain_ollama import ChatOllama
load_dotenv()
import json

In [154]:
llm = ChatGroq(model_name="Llama3-8b-8192")
import requests

def get_weather(latitude, longitude):
    """This is a publically available API that returns the weather for a given location."""
    response = requests.get(
        f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m"
    )
    data = response.json()
    return data["current"]

def tool(fun_name, description, parameters):
    return {
        "name": fun_name,
        "description": description,
        "parameters": parameters
    }
    

def get_output(llm,input_text, tools):
    prompt = f"""You are helpfull assistant that will help the users question and analyse it. if it requires the function then it should use that tool.try to find parameters and then use the tool to get the output. 
    Question: {input_text}
    tools: {tools}
    """
    ans = llm.invoke(prompt)
    return ans.content




In [155]:
def get_weather(latitude, longitude):
    """This is a publically available API that returns the weather for a given location."""
    response = requests.get(
        f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m"
    )
    data = response.json()
    return data["current"]

In [156]:
def summation(number1, number2):
    return sum([number1, number2])

In [157]:
tools = {
    "get_weather": {
        "function": get_weather,
        "description": "Returns weather details for a given latitude and longitude.",
        "parameters": ["latitude", "longitude"]
    },

    "summation": {
        "function": summation,
        "description": "Returns the sum of two numbers.",
        "parameters": ["number1", "number2"]
    }
    
}



In [158]:
arguments = {k: v["function"] for k, v in tools.items()}
arguments

{'get_weather': <function __main__.get_weather(latitude, longitude)>,
 'summation': <function __main__.summation(number1, number2)>}

In [159]:
arguments = {k: v["parameters"] for k, v in tools.items()}
arguments

{'get_weather': ['latitude', 'longitude'], 'summation': ['number1', 'number2']}

In [160]:
{json.dumps({k: v["parameters"] for k, v in tools.items()}, indent=2)}

{'{\n  "get_weather": [\n    "latitude",\n    "longitude"\n  ],\n  "summation": [\n    "number1",\n    "number2"\n  ]\n}'}

In [161]:
{json.dumps({k: v["description"] for k, v in tools.items()}, indent=2)}

{'{\n  "get_weather": "Returns weather details for a given latitude and longitude.",\n  "summation": "Returns the sum of two numbers."\n}'}

In [162]:
from pydantic import BaseModel, Field
from typing import Dict, Any, Optional
import json

from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field


class ToolCallResponse(BaseModel):
    use_tool: bool = Field(..., description="Indicates if a function call is required")
    response: Optional[str] = Field(None, description="Assistant's direct response if no tool is required")
    function: Optional[str] = Field(None, description="Function name to call if needed")
    args: Optional[Dict[str, Any]] = Field(None, description="Arguments for the function call")
    

# Set up a parser + inject instructions into the prompt template.
parser = JsonOutputParser(pydantic_object=ToolCallResponse)

def get_output(llm, input_text, tools):
    tools_json = json.dumps({k: v["description"] for k, v in tools.items()}, indent=2)
    arguments = {k: v["parameters"] for k, v in tools.items()}
    
    template = """
    You are a helpful assistant. Determine if the user's question requires a function call only when needed.
    Apply your knowledge and try to find the parameters required for the function.

    input_text: {input_text}
    Available tools: {tools}
    parameters required for functions:{args}
    
    {format_instructions}
    """

    prompt = PromptTemplate(
        template=template,
        input_variables=["input_text", "tools", "args"],
        partial_variables={"format_instructions": parser.get_format_instructions()},
    )

    chain = prompt | llm | parser

    return chain.invoke({"input_text": input_text, "tools": tools_json, "args": arguments})

In [163]:
a = get_output(llm, "what is the currend whether in mumbai", tools)
a

{'use_tool': True,
 'response': '',
 'function': 'get_weather',
 'args': {'latitude': 18.975, 'longitude': 72.8254}}

In [164]:
function = tools[a["function"]]["function"]
args = a["args"]
output = function(**args)

In [165]:
output

{'time': '2025-03-17T09:30',
 'interval': 900,
 'temperature_2m': 31.9,
 'wind_speed_10m': 12.8}

## Final Code

In [168]:
from pydantic import BaseModel, Field
from typing import Dict, Any, Optional
import json

from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

def get_weather(latitude, longitude):
    """This is a publically available API that returns the weather for a given location."""
    response = requests.get(
        f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m"
    )
    data = response.json()
    return data["current"]

def summation(number1, number2):
    return sum([number1, number2])

tools = {
    "get_weather": {
        "function": get_weather,
        "description": "Returns weather details for a given latitude and longitude.",
        "parameters": ["latitude", "longitude"]
    },

    "summation": {
        "function": summation,
        "description": "Returns the sum of two numbers.",
        "parameters": ["number1", "number2"]
    }
    
}

class ToolCallResponse(BaseModel):
    use_tool: bool = Field(..., description="Indicates if a function call is required")
    response: Optional[str] = Field(None, description="Assistant's direct response if no tool is required")
    function: Optional[str] = Field(None, description="Function name to call if needed")
    args: Optional[Dict[str, Any]] = Field(None, description="Arguments for the function call")
    

# Set up a parser + inject instructions into the prompt template.
parser = JsonOutputParser(pydantic_object=ToolCallResponse)

def get_output(llm, input_text, tools):
    tools_json = json.dumps({k: v["description"] for k, v in tools.items()}, indent=2)
    arguments = {k: v["parameters"] for k, v in tools.items()}
    
    template = """
    You are a helpful assistant. Determine if the user's question requires a function call only when needed.
    Apply your knowledge and try to find the parameters required for the function.

    input_text: {input_text}
    Available tools: {tools}
    parameters required for functions:{args}
    
    {format_instructions}
    """

    prompt = PromptTemplate(
        template=template,
        input_variables=["input_text", "tools", "args"],
        partial_variables={"format_instructions": parser.get_format_instructions()},
    )

    chain = prompt | llm | parser    
    a = chain.invoke({"input_text": input_text, "tools": tools_json, "args": arguments})

    function = tools[a["function"]]["function"]
    args = a["args"]
    output = function(**args)

    return output

get_output(llm, "what is the currend whether in mumbai", tools)

{'time': '2025-03-17T09:30',
 'interval': 900,
 'temperature_2m': 31.9,
 'wind_speed_10m': 12.8}

In [32]:

print(get_output(llm, "Who is the PM of india", tools))


{'response': 'AI is a fascinating field that involves machine learning and deep learning.'}
